In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

print("Setup Complete")

Setup Complete


In [2]:
# Read the samples and convert to array
sample = 5000
mimic_normal_ppg_filepath = "../golden_data/mimic_normal_data_100hz.csv"
mimic_normal_ppg_data = pd.read_csv(mimic_normal_ppg_filepath, nrows = sample)

signal = np.asarray(mimic_normal_ppg_data, dtype = np.float64)

print(signal)

[[0.400824  ]
 [0.38522001]
 [0.37588673]
 ...
 [0.35889018]
 [0.35043673]
 [0.34275404]]


In [3]:
# Enhance signal

print(signal)

for i in range(len(signal)):
  signal[i] *= signal[i]

print(signal)

[[0.400824  ]
 [0.38522001]
 [0.37588673]
 ...
 [0.35889018]
 [0.35043673]
 [0.34275404]]
[[0.16065988]
 [0.14839446]
 [0.14129083]
 ...
 [0.12880216]
 [0.1228059 ]
 [0.11748033]]


In [4]:
# Choose the Windows Size W1, W2 in TERMA framework
w_cycle =  55  # W1
w_evt = 9  #W2

ma_cycle = np.zeros_like(signal, dtype = np.float64)
ma_evt = np.zeros_like(signal, dtype = np.float64)

mean_signal = 0

# Calculate the Event Duration Moving Average
for i in range((w_evt - 1) // 2, len(signal) - ((w_evt - 1) // 2)):
    for j in range(-(w_evt - 1) // 2, (w_evt - 1) // 2):
        ma_evt[i] += signal[i + j]
    ma_evt[i] /= w_evt

# Calculate the Event Cycle Moving Average
for i in range((w_cycle - 1) // 2, len(signal) - ((w_cycle - 1) // 2)):
    for j in range(-(w_cycle - 1) // 2, (w_cycle - 1) // 2):
        ma_cycle[i] += signal[i + j]
    ma_cycle[i] /= w_cycle

# Calculate the mean of signal
for i in range(len(signal)):
    mean_signal += signal[i]
mean_signal /= len(signal)
mean = np.full(len(signal), mean_signal)

In [5]:
# Calculate the Threshold for detect Block of Interest
beta = 0.05
alpha = np.zeros_like(signal, dtype = np.float64)
threshold_1 = np.zeros_like(signal, dtype = np.float64)
block_of_interest = np.zeros_like(signal, dtype = np.float64)

for i in range(len(signal)):
  threshold_1[i] = ma_cycle[i] + beta * mean[i]

print(threshold_1)

# Generate the Block of Interest
for i in range(len(signal)):
  if (ma_evt[i] > threshold_1[i]) and (ma_cycle[i] != 0):
    block_of_interest[i] = 1
  else:
    block_of_interest[i] = 0

[[0.0117032]
 [0.0117032]
 [0.0117032]
 ...
 [0.0117032]
 [0.0117032]
 [0.0117032]]


In [6]:
# Peak detector
start_block = 0
stop_block = 0
peak = 0
peak_index = 0
peak_index_arr = np.zeros_like(signal, dtype = np.float64)
peak_arr = np.zeros_like(signal, dtype = np.float64)
block_num = 0

for i in range(len(block_of_interest) - 1):
  if (block_of_interest[i + 1] - block_of_interest[i]) == 1:
    start_block = i
  if (block_of_interest[i] - block_of_interest[i + 1]) == 1:
    stop_block = i
    if (stop_block - start_block) >= w_evt:
      peak = signal[start_block]
      for i in range(start_block, stop_block + 1):
        if signal[i] > peak:
          peak = signal[i]
          peak_index = i
      peak_arr[block_num] = peak
      peak_index_arr[block_num] = peak_index
      block_num += 1

In [7]:
plt.figure(figsize = (20, 6))

# sns.lineplot(data=signal, x='Value', y='Samples', linestyle='dashed',color='green', label='Filter Golden Data')
# sns.lineplot(data=ma_evt_duration, linestyle='dashed',color='green', label='MA Event Duration')
# sns.lineplot(data=ma_evt_cycle, linestyle='dotted', color='red', label='MA Event Cycle')

x = np.arange(len(signal))

plt.plot(x, signal, linewidth = 2, color = 'red', label = 'Mimic Normal PPG Data')
plt.plot(x, ma_evt, linewidth = 1, color = 'green', linestyle = '--', label = 'MA Event Duration W1')
plt.plot(x, ma_cycle, linewidth = 1, color = 'blue', linestyle = '-.', label = 'MA Event Cycle W2')
plt.plot(x, mean, linewidth = 1, color = 'black', linestyle = '-.', label = 'Mean value')
plt.plot(x, block_of_interest, linewidth = 1, color = 'brown', linestyle = ':', label = 'Block of interest')


plt.scatter(peak_index_arr, peak_arr, s = 50, marker = 'x', c = 'black')

plt.legend()
plt.show()
